In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


soup = BeautifulSoup(page.content, "html.parser")

URL = "https://www.journaldunet.com/business/salaire/index/departements"
page = requests.get(URL)


result = soup.find_all("ul", class_ = "bloc size1of2 odSquareList" )

hrefs = []

for x in result:
    hrefs += x.find_all("a", href = True)


for link in hrefs:
    url = link['href']
    print(url)



# df = pd.DataFrame(data={'store_name':[],'adress':[],'lat':[],'lon':[]})

# soup = BeautifulSoup(page.content, "html.parser")
# results = soup.find(id="default")
# reults_2 = results.find_all("section", class_="lojas-mapa")

# hrefs = []
# for result in reults_2:
#     hrefs += result.find_all("a",href=True)
    
# for link in hrefs:
#     url = "https://www.leroymerlin.pt" + link['href']
#     print(url)
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, "html.parser")
#     results = soup.find(id="default")
#     reults_2 = results.find("section", class_="block-card-conteudo-2")
#     if reults_2 is None:
#         print('Check this URL above!\n\n')
#         continue
#     reults_3 = reults_2.find_all("div", class_="card-body")
#     for job_result in reults_3:
#         title = job_result.find("h2")
#         if title is None:
#             print('Check this URL above!\n\n')
#             continue
#         if title.text != 'LOJA':
#             continue
#         details = job_result.find_all('p')
#         adress = ''
#         for i in range(0,len(details)):
#             if details[i].text[:11].strip().lower()!='coordenadas':
#                 adress += details[i].text
#                 continue
#             coordinates = details[i].text
#             if not ':' in coordinates:
#                 print('Check this URL above!\n\n')
#                 lat = '0'
#                 lon = '0'
#                 continue
#             lat,lon = coordinates.split(':')[1].split(',')  
#             break
#         df = df.append({'store_name':url.split('/')[-1],'adress':adress.strip(), 'lat':lat.strip(), 'lon':lon.strip()},ignore_index=True)


https://www.journaldunet.com/business/salaire/ain/departement-01
https://www.journaldunet.com/business/salaire/aisne/departement-02
https://www.journaldunet.com/business/salaire/allier/departement-03
https://www.journaldunet.com/business/salaire/alpes-de-haute-provence/departement-04
https://www.journaldunet.com/business/salaire/alpes-maritimes/departement-06
https://www.journaldunet.com/business/salaire/ardeche/departement-07
https://www.journaldunet.com/business/salaire/ardennes/departement-08
https://www.journaldunet.com/business/salaire/ariege/departement-09
https://www.journaldunet.com/business/salaire/aube/departement-10
https://www.journaldunet.com/business/salaire/aude/departement-11
https://www.journaldunet.com/business/salaire/aveyron/departement-12
https://www.journaldunet.com/business/salaire/bas-rhin/departement-67
https://www.journaldunet.com/business/salaire/bouches-du-rhone/departement-13
https://www.journaldunet.com/business/salaire/calvados/departement-14
https://www.